This notebook is based on the tensorflow docs: 


https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

adapted for a regression problem.

In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler

In [21]:
a = tuple(range(1, 6))
type(a)
a

(1, 2, 3, 4, 5)

In [19]:
file = "https://raw.githubusercontent.com/la-escamilla/codigos/master/rho_tanh15_phy_rho_corr%2BCBAO%2BHD%2BSN_nested_dynesty_multi_1.txt"

split = 0.8
epochs = 500

cols = tuple(range(1, 19))
data = np.loadtxt(file, usecols=cols)
params = data[:, 1:]
logL = data[:, 0]


nparams = len(params)
randomize = np.random.permutation(nparams)
params = params[randomize]
logL = logL[randomize]

ntrain = int(split * nparams)
indx = [ntrain]

params_training, params_testing = np.split(params, indx)
logL_training, logL_testing = np.split(logL, indx)


# # create scaler
# scaler = StandardScaler()
# # # scaler = MinMaxScaler(feature_range=(-1,1))
# # # fit scaler on data
# scaler.fit(params_training)
# # # apply transform
# params_training = scaler.transform(params_training)
# params_testing = scaler.transform(params_testing)

print("Total len dataset {}," 
      "len training set {}, len test set {}".format(nparams, 
                                                 len(params_training), len(params_testing)))



Total len dataset 40039,len training set 32031, len test set 8008


In [20]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0,
                                   patience=10,
                                   restore_best_weights=True)
                                   ]

n_cols = params_training.shape[1]

In [31]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200, 250, 300]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200, 250, 300]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200, 250, 300]))

HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([8, 16, 32, 64]))

In [32]:
# METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
      hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])


In [33]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation=tf.nn.relu, input_shape=(n_cols,)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=tf.nn.relu),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS3], activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )

    
    model.fit(params_training, logL_training, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)


    return loss
    

In [34]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [35]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                hparams = {
                    HP_NUM_UNITS1: num_units1,
                    HP_NUM_UNITS2: num_units2,
                    HP_NUM_UNITS3: num_units3,
                    HP_BATCHSIZE: batch_size
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams)
                session_num += 1

                    

--- Starting trial: run-0
{'num_units1': 50, 'num_units2': 50, 'num_units3': 50, 'batch_size': 8}
Epoch 1/500
3766/4004 [===========================>..] - ETA: 0s - loss: 492334.9375 - mean_squared_error: 492334.9375

KeyboardInterrupt: 

In [18]:
%tensorboard --logdir logs/hparam_tuning